# Derivativos de Taxa de Juros
## Modelo de Kalotay-Williams-Fabozzi

Renato Mori, Uirá Caiado. 19 de Junho, 2016

**Resumo**

*Neste projeto vamos implementar um modelo de um fator chamado [Kalotay-Williams-Fabozzi](https://en.wikipedia.org/wiki/Short-rate_model#One-factor_short-rate_models), onde apenas um único fator estocástico - a taxa de juros instantânea - determina a evolução de toda estrutura a termo da taxa de juros. Vamos descrever brevemente o modelo, detalhar sua implementação, estimar seus parâmetros e demonstrar sua utilização para precificação de alguns títulos que sejam possíveis de compara com preços de mercado.*

## 1. Introdução

Nesta sessão vamos introduzir o modelo discutido neste trabalho e descrever o problema que será abordado aqui.


### 1.1. Modelagem da Curva de Juros

Como colocado por Fabozzi, a Taxa de Juros frequentemente é modelada usando equações diferenciais estocásticas([SDEs](https://en.wikipedia.org/wiki/Stochastic_differential_equation) é a abreviação em inglês). Os modelos mais comumente utilizados são os modelos de um ou dois fatores (estocástios), sendo que o de um usa uma SDE para representar a taxa de juros de curto prazo e a segunda representa tanto a taxa de curto como a de longo prazo. Na dinâmica escolhida para o modelo que deve ser refletidas as características desejadas para a curva, como reversão a média e volatilidade dependente do nível da taxa.

Existe duas abordagens para na utilizaçtão de equações estocásticas na modelagem da curva a termo de juros: modelos de equilibrio e modelos de não arbitragem, sendo que ambos são usados para precificar *Bonds* e derivativos sobre taxa de juros.

Modelos de equilíbrio começam com uma SDE e desenvolvem uma estrutura que não necessariamente correspondem aos preços observados no mercado. Alguns exemplos de modelos desta classe são o Vasicek, Brenna and Schwartz e o Cox, Ingersoll and Ross.

Os modelos de não arbitragem também utilizam uma SDE, porém usam os preços de mercado para gerar os caminhos possíveis para taxa de juros. Esta estrutura segue as características impostas pela SDE as taxas de juros encontradas, quando usadas para precificar um bond qualquer, resultam na mesma taxa de juros observada no mercado. Exemplos são o Ho and Lee, Black, Derman and Toy e o Hull and White.

Os modelos de não arbitragem geralmente são escolhidos para precificação de Bonds e derivativos pois garantem que as taxas atuais correspondam ao mercado. Este trabalgho explorará um modelo de não arbitragem chamado Kalotay-Williams-Fabozzi.

### 1.2. Modelo Kalotay-Williams-Fabozzi

O modelo Kalotay-Williams-Fabozzi (KWF) assume que as mudanças na taxa de juros de curto prazo segue um certo [processo estocástico](https://en.wikipedia.org/wiki/Stochastic_process) que pode ser descrito pela seguinte SDE:

$$ \mathrm{d} \ln(r) = \theta(t) \mathrm{d} t + \sigma\mathrm{d}Z$$

Onde $Z$ é um processo de [Wiener](https://en.wikipedia.org/wiki/Wiener_process), $\theta$ é o *drift* do processo e $\sigma$ é a volatilidade da primeira diferença das taxas de curto prazo.  Como o que a equação descreve é a dinâmica do **logarítimo natural** de taxa $r$ e $W$ tem distribuição normal, o $\ln(r)$ terá distribuição normal e o $r$, log normal. Desta maneira, embora $\ln(r)$ possa assumir valores negativos, $r$ não assumirá.

No trabalho original em que o modelo KWF foi apresentado, os autores não incluiram explicitamente o *drift* na dinâmica do processo e, por tanto, também não utilizaremos ele aqui,

## 2. Implementando o Modelo

Nesta sessão detalharemos e implementaremos o modelo utilizando como referência o artigo original e calibraremos ele para que bata com os dados de mercado.

### 2.1. Aproximação da SDE por Árvore Binomial

As equações diferenciais estocásticas podem ter sua solução aproximada numericamente utilizando uma árvore binomial. Este método assume que a taxa de juros de curto prazo pode assumir apenas dois valores no período seguinte da discretização, $r_{u}$ ou $r_{d}$, onde $r_{u} > r_{d}$. Como cada nó da árvore dá origem à dois novos nós, a medida que se aumenta os passos de discretização, o número de nós rapidamente aumenta. Para evitar este crescimento exponencial, é imposto ao modelo que a árvore seja recombinante, ou seja, um movimento de subida seguido por uma queda resulta no mesmo nó de uma queda seguida de uma alta. 

Um movimento de alta na taxa de juros tem probabilidade $q$ de ocorrer, enquanto um de queda tem probabilidade de $1 - q$. Utilizaremos probabilidade de $q=0.5$ para garantir a netralidade a risco na solução da SDE. É importante frisar que isso não implica na probabilidade real da taxa de juros cair. $q$ representa apenas a probabilidade *neutra a risco*. 

In [3]:
# bibliotecas necessarias
import numpy as np
import math
import pandas as pd
import seaborn as sns
import matplotlib.pylab as plt

In [205]:
from collections import defaultdict

class Node(object):
    '''
    '''
    def __init__(self, s_name, node_src=None):
        '''
        '''
        # conta a qtde de subidas e descidas para
        # organizacao de nos posteriormente
        i_d = s_name.count("D") * -1
        i_u = s_name.count("U")
        i_len = len(s_name) - 1
        self.node_idx = "{},{}".format(i_len, i_d+i_u)
        # guarda nome e inicia branch
        self.name = str(s_name)

    def get_childrens(self):
        '''
        '''
        s_name = self.name
        if self.name == "_":
            s_name = ""
        return s_name + 'D',  s_name + 'U'

    def __str__(self):
        '''
        '''
        return self.name

    def __repr__(self):
        '''
        '''
        return self.name
    
    def __eq__(self, other):
        '''
        '''
        if isinstance(other, str):
            i_aux = other.count("D")*-1 + other.count("U")
            i_len = len(other) - 1
            s_aux = "{},{}".format(i_len, i_aux)
            return self.node_idx == s_aux
        return self.node_idx == other.node_idx
    
    def __ne__(self, other):
        '''
        '''
        return not self.__eq__(other)
    
    def __hash__(self):
        '''
        '''
        return self.node_idx.__hash__()


class RecombiningTree(object):
    '''
    A Recombining Tree representation
    '''
    def __init__(self, i_steps):
        '''
        '''
        # inicia variaveis
        self.i_steps = i_steps
        self.set_of_nodes = set([])
        self.d_step = defaultdict(list)
        self.d_step[0] = [Node("_")]
        # constroi arvore
        self.set_branchs(i_steps)
    
    
    def set_branchs(self, i_steps):
        '''
        '''
        # constroi arvore
        for i_step in xrange(1, i_steps):
            for node in self.d_step[i_step-1]:
                s_down, s_up = node.get_childrens()
                self.add_node(i_step, s_down)
                self.add_node(i_step, s_up)

    def add_node(self, i_step, s_name):
        '''
        '''
        node = Node(s_name)
        if node not in self.set_of_nodes:
            self.set_of_nodes.add(node)
            self.d_step[i_step].append(node)

    def __str__(self):
        s_res = ''
        i_steps = min(5, self.i_steps)
        # cria string para formatar resultados
        s_aux = ''
        for idx in xrange(i_steps):
            s_aux += '{}\t'
        l_rtn = [s_aux] * i_steps
        # cria string de saida
        for k in xrange(i_steps, 0, -1):
            for d in self.d_step[k]:
                l_rtn[k] = d
        return s_res[:-1]

In [210]:

s_res = ''
i_steps = min(5, self.i_steps)

s_aux = ''
for idx in xrange(i_steps):
    s_aux += '{}\t'
l_rtn = [s_aux] * i_steps

In [213]:
 range(5, 1, -1)

[5, 4, 3, 2]

In [221]:
# cria string de saida
for k in xrange(i_steps-1, 0, -1):
    for d in self.d_step[k]:
        pass

In [218]:
self.d_step[k]

[D, U]

In [219]:
k

1

In [220]:
l_rtn

['{}\t{}\t{}\t{}\t{}\t', U, UU, UUU, UUUU]

In [203]:
s_aux

'{}\t{}\t{}\t{}\t{}\n'

In [201]:
s_aux

'{}\t{}\t{}\t{}\t{}\t'

In [207]:
self = x

In [193]:
x = RecombiningTree(6)

In [194]:
import pprint
pprint.pprint(dict(x.d_step))

{0: [_],
 1: [D, U],
 2: [DD, DU, UU],
 3: [DDD, DDU, DUU, UUU],
 4: [DDDD, DDDU, DDUU, DUUU, UUUU],
 5: [DDDDD, DDDDU, DDDUU, DDUUU, DUUUU, UUUUU]}


In [160]:
x.d_step

defaultdict(list,
            {0: [_],
             1: [D, U],
             2: [DD, DU, UD, UU],
             3: [DDD, DDU, DUD, DUU, UDD, UDU, UUD, UUU]})

In [166]:
x.d_step[2][0].node_idx

'1,-2'

In [168]:
"DDD" in x.set_of_nodes

False

In [157]:
"1,-2" in x.set_of_nodes

False

In [170]:
x.__hash__()

287732093

In [153]:
list(x.set_of_nodes)[1].node_idx

'1,-2'

In [144]:
"1,0" in x.set_of_nodes

False

In [121]:
d={}
d[x.d_step[2][0]]=1

In [125]:
d[x.d_step[2][1]]=1
d[x.d_step[2][2]]=1

In [127]:
d[x.d_step[2][0]]=1

In [128]:
d

{UD: 1, DD: 1, DU: 1}

In [53]:
l1 = [node_U]
l2 = [node_U]

In [66]:
d = {node_U: 1} 

In [67]:
my_set  = set([node_U])

In [71]:
d.get("U")

1

In [69]:
d

{U: 1}

In [80]:
node_UDU == "UUD"

True

In [56]:
l2[0].name

'X'

In [54]:
node_U.name="X"

In [78]:
my_tree = RecombiningTree()

node_root = Node("_")
node_U = Node("U")
node_UU = Node("UU")
node_UD = Node("UD")
node_UDU = Node("UDU")
node_UDD = Node("UDD")
node_UUD = Node("UUD")
node_D = Node("D")
my_tree.addNode(node_root)
my_tree.addNode(node_U)
my_tree.addNode(node_D)

In [48]:
print my_tree

In [50]:
self = my_tree

In [51]:
print self

In [52]:
self.branchs

{_: [], U: [], D: []}

In [11]:
class Tree(object)
    '''
    '''
    class __init__(self):
        '''
        '''
        self.root = None

In [9]:
tree = Digraph()

In [10]:
tree.addNode(x)

In [8]:


class Edge(object):
    def __init__(self, src, dest):
        self.src = src
        self.dest = dest
    def getSource(self):
        return self.src
    def getDestination(self):
        return self.dest
    def __str__(self):
        return '{0}->{1}'.format(self.src, self.dest)

class Digraph(object):
    """
    A directed graph
    """
    def __init__(self):
        # A Python Set is basically a list that doesn't allow duplicates.
        # Entries into a set must be hashable (where have we seen this before?)
        # Because it is backed by a hashtable, lookups are O(1) as opposed to the O(n) of a list (nifty!)
        # See http://docs.python.org/2/library/stdtypes.html#set-types-set-frozenset
        self.nodes = set([])
        self.edges = {}
    def addNode(self, node):
        if node in self.nodes:
            # Even though self.nodes is a Set, we want to do this to make sure we
            # don't add a duplicate entry for the same node in the self.edges list.
            raise ValueError('Duplicate node')
        else:
            self.nodes.add(node)
            self.edges[node] = []
    def addEdge(self, edge):
        src = edge.getSource()
        dest = edge.getDestination()
        if not(src in self.nodes and dest in self.nodes):
            raise ValueError('Node not in graph')
        self.edges[src].append(dest)
    def childrenOf(self, node):
        return self.edges[node]
    def hasNode(self, node):
        return node in self.nodes
    def __str__(self):
        res = ''
        for k in self.edges:
            for d in self.edges[str(k)]:
                res = '{0}{1}->{2}\n'.format(res, k, d)
        return res[:-1]

In [24]:
print "\t U \n_\n\tD"

	U
_
	D


In [172]:

class Node(object):
    def __init__(self, name):
        self.name = str(name)
    def getName(self):
        return self.name
    def __str__(self):
        return self.name
    def __repr__(self):
        return self.name
    def __eq__(self, other):
        return self.name == other.name
    def __ne__(self, other):
        return not self.__eq__(other)
    def __hash__(self):
        # Override the default hash method
        # Think: Why would we want to do this?
        return self.name.__hash__()

class Edge(object):
    def __init__(self, src, dest):
        self.src = src
        self.dest = dest
    def getSource(self):
        return self.src
    def getDestination(self):
        return self.dest
    def __str__(self):
        return '{0}->{1}'.format(self.src, self.dest)

class Digraph(object):
    """
    A directed graph
    """
    def __init__(self):
        # A Python Set is basically a list that doesn't allow duplicates.
        # Entries into a set must be hashable (where have we seen this before?)
        # Because it is backed by a hashtable, lookups are O(1) as opposed to the O(n) of a list (nifty!)
        # See http://docs.python.org/2/library/stdtypes.html#set-types-set-frozenset
        self.nodes = set([])
        self.edges = {}
    def addNode(self, node):
        if node in self.nodes:
            # Even though self.nodes is a Set, we want to do this to make sure we
            # don't add a duplicate entry for the same node in the self.edges list.
            raise ValueError('Duplicate node')
        else:
            self.nodes.add(node)
            self.edges[node] = []
    def addEdge(self, edge):
        src = edge.getSource()
        dest = edge.getDestination()
        if not(src in self.nodes and dest in self.nodes):
            raise ValueError('Node not in graph')
        self.edges[src].append(dest)
    def childrenOf(self, node):
        return self.edges[node]
    def hasNode(self, node):
        return node in self.nodes
    def __str__(self):
        res = ''
        for k in self.edges:
            for d in self.edges[str(k)]:
                res = '{0}{1}->{2}\n'.format(res, k, d)
        return res[:-1]


In [173]:
set_test = set([Node("a"), Node("a"), Node("b")])

In [174]:
set_test

{a, b}

In [176]:
Node("a") in set_test

True

In [13]:
class Node:
    def __init__(self, val):
        self.l = None
        self.r = None
        self.v = val

class Tree:
    def __init__(self):
        self.root = None

    def getRoot(self):
        return self.root

    def add(self, val):
        if(self.root == None):
            self.root = Node(val)
        else:
            self._add(val, self.root)

    def _add(self, val, node):
        if(val < node.v):
            if(node.l != None):
                self._add(val, node.l)
            else:
                node.l = Node(val)
        else:
            if(node.r != None):
                self._add(val, node.r)
            else:
                node.r = Node(val)

    def find(self, val):
        if(self.root != None):
            return self._find(val, self.root)
        else:
            return None

    def _find(self, val, node):
        if(val == node.v):
            return node
        elif(val < node.v and node.l != None):
            self._find(val, node.l)
        elif(val > node.v and node.r != None):
            self._find(val, node.r)

    def deleteTree(self):
        # garbage collector will do this for us. 
        self.root = None

    def printTree(self):
        if(self.root != None):
            self._printTree(self.root)

    def _printTree(self, node):
        if(node != None):
            self._printTree(node.l)
            print str(node.v) + ' '
            self._printTree(node.r)

#     3
# 0     4
#   2      8
tree = Tree()
tree.add(3)
tree.add(4)
tree.add(0)
tree.add(8)
tree.add(2)
tree.printTree()
print (tree.find(3)).v
print tree.find(10)
tree.deleteTree()
tree.printTree()

0 
2 
3 
4 
8 
3
None


In [14]:
tree = Tree()
tree.add(3)
tree.add(4)
tree.add(0)
tree.add(8)
tree.add(2)

In [16]:
tree.printTree()

0 
2 
3 
4 
8 


#### TODO:
- Apresentar os principais pontos da abordagem (hipóteses, limitações e etc)
- Implementar
- detalhar o processo de calibração (juros e vol)
- Detalhar a determiação dos parâmetros do modelo ($\sigma$)
- Checar se tem base de dados de opções (Call Européia) de Juros
- Levantar os preços histórico dos títulos: NTB...
- Demonstrar a precificação de 2 derivativos mais complexos (opcional compara com mercado)


In [1]:
[1, 2, 3, 4, 5]

[1, 2, 3, 4, 5]

In [ ]:
{1: [1], 2:[2, 2], 3:[3, 3, 3]}

In [ ]:
class kwf_model(object):
    '''
    '''
    def __init__(self, l_prices, l_maturity, f_sigma, ll_cupon=[])
        '''
        Implements the kwf model
        :param l_prices: list. Market Prices
        :param l_maturity: list. The maturirity in year of the contracts
        :param f_sigma. float. The volatility of the 
        :*ll_cupon: list of lists. The cupons of each instrument
        '''
        self.l_prices = l_prices
        self.l_maturity = l_maturity
        self.f_sigma = f_sigma
        sefl.ll_cupon = ll_cupon
        self.d_forward = self.fit_curve()

    def fit_curve(self):
        '''
        Return the forwards that fit the market curve
        '''
        raise NotImplementedError()
        return d_rtn

    
class Bond(object):
    '''
    '''
    def __init__(self, f_cupon, f_face_value):
        '''
        '''
        raise NotImplementedError()

    def get_value(self, o_kwf_model):
        '''
        '''
        raise NotImplementedError()
        return f_rtn


In [ ]:
l_prices = [0.035, 0.04, 0.045]
l_maturity = [1., 2., 3.]
f_sigma = 0.1


my_curve = kwf_model(l_prices, l_maturity, f_sigma)

In [ ]:
mybond = Bond(0.045, 100.)
mybond.get_value(my_curve)  # retorna preco

## 2. Bla

bla bla

### 2.1. Bla

bla bla

## 3. Conclusão

bla bla

## 4. Últimas Considerações

bla bla

*Style notebook and change matplotlib defaults*

In [3]:
#loading style sheet
from IPython.core.display import HTML
HTML(open('ipython_style.css').read())

IOError: [Errno 2] No such file or directory: ''

In [2]:
#changing matplotlib defaults
%matplotlib inline
import seaborn as sns
sns.set_palette("deep", desat=.6)
sns.set_context(rc={"figure.figsize": (8, 4)})
sns.set_style("whitegrid")
sns.set_palette(sns.color_palette("PuBuGn_d", 10))